<a href="https://colab.research.google.com/github/FawwazMayda/BangkitPlantPathology/blob/master/BaselineVGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!kaggle -v

Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.0.2)
Processing /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01/kaggle-1.5.6-py3-none-any.whl
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.6/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 149, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
import os
os.environ['KAGGLE_USERNAME']="medias"
os.environ['KAGGLE_KEY']="2751bb484f8f6ddff17b7a812d26367a"

In [0]:
!kaggle competitions download -c plant-pathology-2020-fgvc7

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.6/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 149, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
!unzip plant-pathology-2020-fgvc7.zip

Archive:  plant-pathology-2020-fgvc7.zip
replace images/Test_0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace images/Test_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
from tqdm import tqdm
import cv2
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [0]:
submission = pd.read_csv("sample_submission.csv")
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [0]:
train.head()

In [0]:
test.sample()

In [0]:
submission.sample(5)

In [0]:
!ls

In [12]:
train_img = []
path = "./images"
for im in tqdm(train['image_id']):
  im = im+".jpg"
  fnpath = os.path.join(path,im)
  img = cv2.imread(fnpath)
  img = cv2.resize(img,(224,224))
  img = img.astype('float32')
  train_img.append(img)


100%|██████████| 1821/1821 [00:59<00:00, 30.74it/s]


In [13]:
test_img = []
path = "./images"
for im in tqdm(test['image_id']):
  im = im+".jpg"
  fnpath = os.path.join(path,im)
  img = cv2.imread(fnpath)
  img = cv2.resize(img,(224,224))
  img = img.astype('float32')
  test_img.append(img)

100%|██████████| 1821/1821 [00:59<00:00, 30.40it/s]


In [0]:
train_label = train.iloc[:,1:]
train_img = np.array(train_img)
test_img = np.array(test_img)

In [15]:
print(train_img.shape)
print(train_label.shape)

(1821, 224, 224, 3)
(1821, 4)


In [16]:
test_img.shape

(1821, 224, 224, 3)

Creating Data generator

In [0]:
import tensorflow as tf

In [0]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function = tf.keras.applications.vgg16.preprocess_input,
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        )
        
datagen.fit(train_img)

In [24]:
base = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet',input_shape=(224,224,3),pooling='avg')
base.trainable = False
model = tf.keras.models.Sequential()
model.add(base)
model.add(tf.keras.layers.Dense(100,'relu'))
model.add(tf.keras.layers.Dense(4,'softmax'))

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)

    

callbacks = [reduce_lr]
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(datagen.flow(train_img,train_label),batch_size=32,epochs=20,callbacks=callbacks,steps_per_epoch=200
          )
#model.fit_generator(datagen.flow(train_img, train_label, batch_size=32),epochs=20,callbacks=callbacks)

Epoch 1/20
200/200 [==============================] - 73s 365ms/step - loss: 0.9624 - accuracy: 0.6688 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 74s 368ms/step - loss: 0.6068 - accuracy: 0.7772 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 73s 365ms/step - loss: 0.5224 - accuracy: 0.7997 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 74s 371ms/step - loss: 0.4515 - accuracy: 0.8292 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 73s 365ms/step - loss: 0.4200 - accuracy: 0.8479 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 73s 365ms/step - loss: 0.3855 - accuracy: 0.8572 - lr: 0.0010
Epoch 7/20
200/200 [==============================] - 73s 366ms/step - loss: 0.3469 - accuracy: 0.8692 - lr: 0.0010
Epoch 8/20
200/200 [==============================] - 73s 365ms/step - loss: 0.2987 - accuracy: 0.8893 - lr: 0.0010
Epoch 9/20
200/200 [==============================] - 72s 359ms/step - l

In [0]:
y_pred=model.predict(test_img)
submission.loc[:,'healthy':'scab']=y_pred

In [28]:
print(submission.head())
submission.to_csv('submission.csv',index=False)

  image_id       healthy  multiple_diseases          rust          scab
0   Test_0  9.784098e-01       1.656282e-06  2.055527e-02  1.033321e-03
1   Test_1  9.996955e-01       5.453593e-07  1.234908e-04  1.804094e-04
2   Test_2  6.865301e-07       4.188961e-02  5.610644e-01  3.970453e-01
3   Test_3  1.000000e+00       2.558612e-14  6.322251e-09  1.006801e-09
4   Test_4  2.001811e-05       5.931570e-11  9.335598e-01  6.642015e-02


In [40]:
!kaggle competitions submit -c plant-pathology-2020-fgvc7 -f submission.csv -m "LB Sub"

100% 165k/165k [00:08<00:00, 20.9kB/s]
Successfully submitted to Plant Pathology 2020 - FGVC7